## Demonstration: augmentation of input chips and labels

This notebook demonstrates augmentation of input chips and labels. This is a device that is used in training of deep learning models to 'stretch' limited data so that the model sees training samples with slight differences, rather than the same samples over and over. 

The notebook demonstrates setting up the ramp data generator for training, then applying image transformations randomly to the images, and displaying the results. Thus, you can see what augmentation is actually doing to the images. 

Augmentations were used successfully in the [Google building footprints over Africa project](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwiG36Xji5z4AhXlAJ0JHdi_BKwQFnoECAUQAQ&url=https%3A%2F%2Fsites.research.google%2Fopen-buildings%2F&usg=AOvVaw0f_y7vp7CYXhnQFgB82uJR) to improve building model extraction quality. The specific image transformations used in that project were:

- random cropping of images
- horizontal and vertical image flipping
- random rotations
- random modifications to image hue, brightness and contrast. 

They found that random modifications to color helped the model deal better with variable image quality and atmospheric conditions. 

The chip and mask images at the bottom of this notebook show the effects of image augmentations of the above types.  

In [ ]:
%matplotlib inline

In [ ]:
import tensorflow as tf 
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # only print errors
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# set up logging
import logging
logging.basicConfig(level = logging.INFO)

### Import ramp code.

In [ ]:
import sys
RAMP_HOME = os.environ["RAMP_HOME"]
from ramp.data_mgmt.data_generator import training_batches_from_gtiff_dirs

In [ ]:
train_base = os.path.join(RAMP_HOME, 'ramp-code/notebooks/sample-data/training_data')
image_dir = os.path.join(train_base, "chips")
mask_dir = os.path.join(train_base, "multimasks")
batch_size = 12
input_image_size = (256,256)
output_image_size = (256,256)

In [ ]:
train_batches = training_batches_from_gtiff_dirs(image_dir, 
                                                 mask_dir, 
                                                 batch_size, 
                                                 input_image_size, 
                                                 output_image_size)

In [ ]:
chips = None
masks = None
for items in train_batches.take(1):
    chips = items[0]
    masks = items[1]
    break

In [ ]:
def visualize(image, mask, original_image=None, original_mask=None):
    fontsize = 18
    
    if original_image is None and original_mask is None:
        f, ax = plt.subplots(2, 1, figsize=(8, 8))

        ax[0].imshow(image)
        ax[1].imshow(mask)
    else:
        f, ax = plt.subplots(2, 2, figsize=(8, 8))

        ax[0, 0].imshow(original_image)
        ax[0, 0].set_title('Original image', fontsize=fontsize)
        
        ax[1, 0].imshow(original_mask)
        ax[1, 0].set_title('Original mask', fontsize=fontsize)
        
        ax[0, 1].imshow(image)
        ax[0, 1].set_title('Transformed image', fontsize=fontsize)
        
        ax[1, 1].imshow(mask)
        ax[1, 1].set_title('Transformed mask', fontsize=fontsize)


### Define image augmentations  

When random rotation is applied to images, some information at the edges of the image is rotated out of view, and new areas without data are rotated into view. It is important that this new data is identified as nodata (0-0-0) in the image chips, and as non-buildings in the mask chips (0). This is what the 'value' and 'mask_value' arguments to the Rotate transform specify.

In [ ]:
import albumentations as A
from cv2 import BORDER_CONSTANT, INTER_NEAREST

aug = A.Compose([
                A.Rotate(
                    border_mode=BORDER_CONSTANT, 
                    interpolation=INTER_NEAREST, 
                    value=(0.0,0.0,0.0), 
                    mask_value = 0, 
                    p=0.9),
                A.RandomBrightnessContrast(brightness_limit=0.2, 
                    contrast_limit=0.2, 
                    brightness_by_max=True, 
                    p=0.9)
        ])

In [ ]:
new_img = np.zeros(chips.shape)
new_mask = np.zeros(masks.shape)
for ii in range(batch_size):
    the_img = chips[ii,:,:,:]
    the_mask = masks[ii,:,:,0]
    augmented = aug(image=the_img.numpy(), mask=the_mask.numpy())
    new_img[ii,:,:,:] = augmented['image']
    new_mask[ii,:,:,0] = augmented['mask']
    visualize(new_img[ii,:,:,:], new_mask[ii,:,:,0], the_img.numpy(), the_mask.numpy())


##### Created for ramp project, August 2022
##### Author: carolyn.johnston@dev.global